Importing modules

In [2]:
import pandas as pd
import glob

Concatenating all bike rental data into one dataframe

In [3]:
path = '/Users/ethanenkhtur/Documents/Codecademy/bike-rental-starter-kit/data/JC*'

csv_files = glob.glob(path)

dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)
    
df = pd.concat(dataframes, ignore_index=True)

In [9]:
df.head(3)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,361,2016-02-01 00:31:18,2016-02-01 00:37:19,3202,Newport PATH,40.727224,-74.033759,3203,Hamilton Park,40.727596,-74.044247,24393,Subscriber,1975.0,1
1,297,2016-02-01 01:55:05,2016-02-01 02:00:02,3195,Sip Ave,40.730743,-74.063784,3194,McGinley Square,40.725340,-74.067622,24394,Subscriber,1985.0,2
2,1155,2016-02-01 02:40:05,2016-02-01 02:59:20,3183,Exchange Place,40.716247,-74.033459,3210,Pershing Field,40.742677,-74.051789,24676,Subscriber,1976.0,1


How much missing data there is

In [4]:
df.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64

In [16]:
df.dtypes

Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
dtype: object

DataFrame could use some fixing in terms of each column being of right type such as start and stop time.

Let's do that!

In [19]:
df['Start Time'] = pd.to_datetime(df['Start Time'])
df['Stop Time'] = pd.to_datetime(df['Stop Time'])
df.dtypes

Trip Duration                       int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth Year                        float64
Gender                              int64
dtype: object

In [5]:
df[df['User Type'].isna()]

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
8546,943,2016-08-01 09:48:21,2016-08-01 10:04:05,3214,Essex Light Rail,40.712774,-74.036486,3199,Newport Pkwy,40.728745,-74.032108,24609,NaN,1987.0,1
8634,2319,2016-08-01 12:29:23,2016-08-01 13:08:03,3199,Newport Pkwy,40.728745,-74.032108,3184,Paulus Hook,40.714145,-74.033552,24609,NaN,1987.0,1
10848,365,2016-08-03 10:46:37,2016-08-03 10:52:43,3214,Essex Light Rail,40.712774,-74.036486,3202,Newport PATH,40.727224,-74.033759,24724,NaN,1987.0,1
13241,396,2016-08-05 10:20:57,2016-08-05 10:27:34,3214,Essex Light Rail,40.712774,-74.036486,3202,Newport PATH,40.727224,-74.033759,24530,NaN,1987.0,1
13385,581,2016-08-05 14:06:10,2016-08-05 14:15:51,3202,Newport PATH,40.727224,-74.033759,3183,Exchange Place,40.716247,-74.033459,24668,NaN,1987.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234992,141,2016-05-13 09:25:49,2016-05-13 09:28:11,3214,Essex Light Rail,40.712774,-74.036486,3183,Exchange Place,40.716247,-74.033459,24436,NaN,1987.0,1
235566,126,2016-05-14 12:13:39,2016-05-14 12:15:45,3183,Exchange Place,40.716247,-74.033459,3214,Essex Light Rail,40.712774,-74.036486,24437,NaN,1987.0,1
235577,142,2016-05-14 12:22:41,2016-05-14 12:25:04,3214,Essex Light Rail,40.712774,-74.036486,3183,Exchange Place,40.716247,-74.033459,24437,NaN,1987.0,1
237296,143,2016-05-17 08:19:54,2016-05-17 08:22:17,3214,Essex Light Rail,40.712774,-74.036486,3183,Exchange Place,40.716247,-74.033459,24606,NaN,1987.0,1


In [15]:
df[(df['Birth Year'] != 1987) & (df['User Type'].isna())]

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
30092,740,2016-08-21 13:19:19,2016-08-21 13:31:40,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,-74.044247,26248,NaN,1980.0,2
31972,574,2016-08-23 10:25:31,2016-08-23 10:35:05,3268,Lafayette Park,40.713464,-74.062859,3192,Liberty Light Rail,40.711242,-74.055701,26266,NaN,1998.0,1
31974,535,2016-08-23 10:26:13,2016-08-23 10:35:08,3268,Lafayette Park,40.713464,-74.062859,3192,Liberty Light Rail,40.711242,-74.055701,24590,NaN,1997.0,1
31997,4125,2016-08-23 11:28:23,2016-08-23 12:37:09,3192,Liberty Light Rail,40.711242,-74.055701,3192,Liberty Light Rail,40.711242,-74.055701,26266,NaN,1975.0,2
32700,973,2016-08-23 23:11:01,2016-08-23 23:27:15,3273,Manila & 1st,40.721651,-74.042884,3192,Liberty Light Rail,40.711242,-74.055701,26267,NaN,1997.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201755,2155,2016-11-27 12:49:31,2016-11-27 13:25:27,3184,Paulus Hook,40.714145,-74.033552,3192,Liberty Light Rail,40.711242,-74.055701,24450,NaN,1960.0,1
201776,1217,2016-11-27 13:28:48,2016-11-27 13:49:05,3192,Liberty Light Rail,40.711242,-74.055701,3192,Liberty Light Rail,40.711242,-74.055701,24450,NaN,1960.0,1
201793,1372,2016-11-27 13:52:07,2016-11-27 14:14:59,3192,Liberty Light Rail,40.711242,-74.055701,3184,Paulus Hook,40.714145,-74.033552,24551,NaN,1960.0,1
202709,835,2016-11-28 20:45:04,2016-11-28 20:59:00,3272,Jersey & 3rd,40.723332,-74.045953,3183,Exchange Place,40.716247,-74.033459,24678,NaN,1991.0,1


In [8]:
df.count()

Trip Duration              247584
Start Time                 247584
Stop Time                  247584
Start Station ID           247584
Start Station Name         247584
Start Station Latitude     247584
Start Station Longitude    247584
End Station ID             247584
End Station Name           247584
End Station Latitude       247584
End Station Longitude      247584
Bike ID                    247584
User Type                  247204
Birth Year                 228585
Gender                     247584
dtype: int64

In [4]:
weather_file = '/Users/ethanenkhtur/Documents/Codecademy/bike-rental-starter-kit/data/newark_airport_2016.csv'

weather_df = pd.read_csv(weather_file)

weather_df.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1
